# Importing the necessary modules

In [93]:
# Utilitaries
from csv import reader
import re
import pandas as pd
import zeep
# Text manipulation

# Data preparation
import nltk
from nltk.corpus import stopwords as stp
import nltk.stem.snowball as snowball
# Machine learning tools
from sklearn.model_selection import train_test_split
# Machine learning classificators
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
# Machine learning model metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import log_loss

# Reading the corpus

In [2]:
df = pd.read_csv('corpus.csv')

In [3]:
df.sample(5)

,author,title,translation,label
928,TVR,"Rezultate parțiale BEC, după centralizarea a 9...","PSD a obținut la alegerile parlamentare 29,64...",1
920,TVR,Marcel Ciolacu: Voi avea o discuție cu colegii...,"Președintele PSD, Marcel Ciolacu, a declarat c...",1
430,Ionut,Cel mai norocos oltean. A găsit o sarma cu pat...,"Chiar dacă a găsit sarmaua cu patru foi, oltea...",0
700,TVR,"Bugetul, dezbătut intens în comisiile de speci...",ACTUALIZARE Bugetul Casei Naţionale de Asigură...,1
342,Eftimie,Fum alb la negocieri. Ludovic Orban fumează ia...,"„Doar un fum să mai trag”, a declarat liderul ...",0


# Data preparation

### Lemmatization
_Source: http://nlptools.info.uaic.ro/WebPosRo/_

In [4]:
wsdl = 'http://nlptools.info.uaic.ro/WebPosRo/PosTaggerRoWS?wsdl'

In [5]:
client = zeep.Client(wsdl=wsdl)

#### Transforming each article using the POS tagger,tokenizing and filtering the stopwords at the same time

In [ ]:
lemmatized_articles_xml=[]

In [6]:
for article in df['translation']:
     lemmatized_articles_xml.append(client.service.parseSentence_XML(article))
    

In [10]:
# # Saving the articles to save execution time
# for article in lemmatized_articles_xml:
#     with open('lemmatized_articles','a+') as f:
#         f.write(article)
#         f.write('\n')

In [62]:
stopwords = stp.words('romanian')
lemmatized_articles_clean=[]
for article in lemmatized_articles_xml:
    article = article.split('\n')
    lemmas = []
    for line in article:
        found = re.search('.*LEMMA="([A-Za-zăâîșț1-9]*)".*', line)
        if found:
            word = found.group(1)
            if word not in stopwords:
                lemmas.append(word)
    lemmatized_articles_clean.append(lemmas)

#### Adding the lemmatized articles to the corpus

In [63]:
df['lemmatized_articles'] = lemmatized_articles_clean

In [67]:
df.sample(5)

,author,title,translation,label,lemmatized_articles
838,TVR,Cum își petrec politicienii Crăciunul,Masa trebuie să fie plină de bucate speciale î...,1,"[masă, trebui, plin, bucată, special, Cristina..."
542,Vasile,Testul COVID făcut de naționala de fotbal arat...,"„COVID n-am găsit, dar până la urmă, mai conte...",0,"[COVID, găsi, conta, adăuga, medic, testa, spo..."
742,TVR,"Diana Șoșoacă, fără sprijinul partidului AUR","DAN TANASĂ, deputat AUR: ""M-a deranjat acel ep...",1,"[dan, tanasă, deputat, aur, deranja, episod, d..."
141,Calin,Ce ghiolban! O specie de țânțar râgâie după ce...,"”Nu sunt adeptul soluțiilor extreme, dar aceas...",0,"[adept, soluție, extrem, specie, țânțar, trebu..."
751,TVR,Orban: Nu s-a luat nicio decizie legată de spo...,El a adăugat că studenţii vor avea în continua...,1,"[adăuga, student, vrea, gratuitate, călătorie,..."


#### Stemming

In [78]:
stemmed_articles = []
stemmer = snowball.SnowballStemmer('romanian')

for article in df['lemmatized_articles']:
    stemmed_words =[]
    for word in article:
        stemmed_words.append(stemmer.stem(word))
        # Appending the string form of the stemmed array because it will
        # be needed in the next step
    stemmed_articles.append(' '.join(stemmed_words))

#### Adding the stemmed articles to the corpus

In [79]:
df['stemmed_articles']=stemmed_articles

In [80]:
df.sample(5)

,author,title,translation,label,lemmatized_articles,stemmed_articles
587,Calin,România a decis să nu se prezinte la meciul cu...,Vestea a fost primită cu optimism de jucătorii...,0,"[veste, primit, optimism, jucător, lot, națion...",vest primit optimist jucat lot național grij v...
398,Vasile,"Validarea lui Ciucu ca primar, blocată în inst...",„Specialiştii noştri atrag atenţia că numele d...,0,"[specialist, atrage, atenție, nume, domn, ciuc...",specialist atrag atenț num domn ciucu asemăn b...
345,Calin,Nea Costel s-a consultat cu liderii de birt și...,„Am un singur mesaj pentru Orban: dă-te-n p… m...,0,"[singur, mesaj, orban, puncte, transmite, nea,...",singur mesaj orban punct transmit nea costel ț...
404,Vasile,Viorica Dăncilă îl acuză pe Iohannis că a inve...,„Nu doar cuvinte ca giroscop sau funambulesc s...,0,"[cuvânt, giroscop, funambulesc, inventat, Ioha...",cuvânt giroscop funambul invent iohannis și ga...
169,Eftimie,Coca-Cola salvează industria ţiţeiului dublând...,"Pe lângă cola, și alte băuturi din gamă vor be...",0,"[cola, și, băutură, gamă, vrea, beneficia, pet...",col și băut gam vre benefic petrol gust putern...


# Preparation for the Machine Learning training

#### Extracting the article labels

In [91]:
label = df.label

#### Splitting the dataset

In [92]:
X_train,X_test,y_train,y_test=train_test_split(df['stemmed_articles'], label, test_size=0.2,shuffle=True)

# Trying different Machine Learning models

In [94]:
nb_classifier = GaussianNB()
nb_classifier.fit(X_train,y_train)

ValueError: could not convert string to float: 'afirmaț însă greu cred întrucât om sin căut cas zon susțin așa sfârșit lum apartament groap guno glin simț miros pucioas și ansamblu rezidențial acolo cinc minut metrou'